In [2]:

path_data = '/data'

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
warnings.filterwarnings("ignore")
import os



import torch
import transformers
from pkg_resources import packaging
from PIL import Image
import tqdm
from pathlib import Path
print("Torch version:", torch.__version__)

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


import urllib.request
import requests
import open_clip
from sentence_transformers import SentenceTransformer, util
from open_clip import get_tokenizer

from open_clip import create_model_from_pretrained
from PIL import Image, ImageFile
import timm

plt.style.use('ggplot')

Torch version: 2.2.1+cu121


In [4]:
data_unfiltered = pd.read_csv("/".join([path_data, "images_v2.csv"]))
#display(data_unfiltered)

In [5]:
len(data_unfiltered)

38479

### Remove subjective topics

In [6]:
subjective_topics = ['Favorite home decorations', 'Favourite item in kitchen', 'Favourite sports clubs', 'How the most loved item is used', 'icons', 'Idols', 'Latest furniture bought', ' looking over the shoulder', 'Most loved item', 'Most loved toy', 'Most played songs on the radio', 'Music idol', 'Next big thing you are planning to buy', 'Playing with most loved toy', 'Thing I dream about having', 'Things I wish I had', 'Using most loved item', 'Youth culture', 'What I wish I could buy']

In [7]:
len(subjective_topics)

19

In [8]:
unique_topics = data_unfiltered['topics'].unique()

In [9]:
Subjective_topic_list = []

for topic in unique_topics:
    topic_ = topic.replace('/ ', ', ')
    topic_split = topic_.split(',')
    for split in topic_split:
        if split in subjective_topics:
            Subjective_topic_list.append(topic)

In [10]:
data = data_unfiltered[~data_unfiltered['topics'].isin(Subjective_topic_list)]
display(data)

,id,country.name,country.id,region.id,type,imageRelPath,topics,place,income
1,5d4bf31ccf0b3a0f3f359814,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f359814/5d4bf31ccf0b3a...,Family snapshots,butoyi,26.994581
2,5d4bf31ccf0b3a0f3f35982a,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f35982a/5d4bf31ccf0b3a...,Cutlery,butoyi,26.994581
3,5d4bf31ccf0b3a0f3f35982e,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f35982e/5d4bf31ccf0b3a...,Family,butoyi,26.994581
4,5d4bf31ccf0b3a0f3f359830,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f359830/5d4bf31ccf0b3a...,Place where eating dinner,butoyi,26.994581
5,5d4bf31dcf0b3a0f3f35983c,Burundi,bi,af,image,assets/5d4bf31dcf0b3a0f3f35983c/5d4bf31dcf0b3a...,Plate of food,butoyi,26.994581
...,...,...,...,...,...,...,...,...,...
38474,5ec4f5513f62767d97a47324,France,fr,eu,image,assets/5ec4f5513f62767d97a47324/5ec4f5513f6276...,Bed,larriere,19671.000000
38475,5ec4f5513f62767d97a47325,France,fr,eu,image,assets/5ec4f5513f62767d97a47325/5ec4f5513f6276...,Bathroom/Toilet,larriere,19671.000000
38476,5ec4f5523f62767d97a47327,France,fr,eu,image,assets/5ec4f5523f62767d97a47327/5ec4f5523f6276...,Armchair,larriere,19671.000000
38477,5ec4f5523f62767d97a47328,France,fr,eu,image,assets/5ec4f5523f62767d97a47328/5ec4f5523f6276...,Armchair,larriere,19671.000000


In [10]:
len(data)

36754

### Split topics

In [15]:
from collections import Counter
list_topics = list(data['topics'])


separate_topics = [t.lower().strip() for topic in list_topics for t in topic.split(",") ]



print(len(separate_topics))
print(Counter(separate_topics))
set_topics = list(set(separate_topics))
print(len(set_topics))

45692
Counter({'visit': 3177, 'toothbrush': 552, 'home': 542, 'street view': 532, 'hand washing': 530, 'family': 521, 'washing clothes/cleaning': 507, 'tv': 494, 'toilet': 493, 'hand palm': 492, 'drinking water': 487, 'kitchen sink': 480, 'tooth paste': 468, 'stove/hob': 444, 'salt': 438, 'bed': 424, 'spices': 419, 'hair brush/comb': 413, 'everyday shoes': 402, 'bedroom': 400, 'front door': 398, 'plate of food': 397, 'power outlet': 389, 'social drink': 385, 'cooking pots': 384, 'floor': 379, 'place where eating dinner': 378, 'nicest shoes': 375, 'lock on front door': 373, 'phone': 373, 'bathroom privacy': 369, 'wardrobe': 363, 'soap for hands and body': 363, 'ceiling': 362, 'hand back': 360, 'wall inside': 359, 'play area': 357, 'dish washing brush/cloth': 351, 'bathroom/toilet': 350, 'refrigerator': 344, 'toilet paper': 343, 'plates': 341, 'wall decoration': 341, 'toys': 340, 'shampoo': 339, 'dish washing soap': 339, 'light sources': 339, 'dish racks': 338, 'kitchen': 335, 'cups/mugs

In [26]:
# map each topic to list of corresponding images
dict_topic2img = {}
for list_topics, image_path in zip(data['topics'], data['imageRelPath']):
    for topic in list_topics.split(","):
        topic = topic.lower().strip()
        if topic not in dict_topic2img:
            dict_topic2img[topic] = set() #### here a set was used in place of list to avoid duplicate where keyword appears twice in a topic
        dict_topic2img[topic].add(image_path)

print(len(dict_topic2img))

270


In [27]:
#sanity checks 

ground_truth_counts = [len(dict_topic2img[i]) for i in dict_topic2img]
print(sum(ground_truth_counts))

45616


In [28]:
topics = dict_topic2img.keys()

# Create default prompts for the text encoder

In [30]:
temp = "This is a photo of "
English_topics = [temp + topic for topic in topics]

In [31]:
#English_topics

In [32]:
topic2english = {}
for topic, prompt in zip(topics, English_topics):
    topic2english[topic] = prompt

In [33]:
topic2english

{'family snapshots': 'This is a photo of family snapshots',
 'cutlery': 'This is a photo of cutlery',
 'family': 'This is a photo of family',
 'place where eating dinner': 'This is a photo of place where eating dinner',
 'plate of food': 'This is a photo of plate of food',
 'table with food': 'This is a photo of table with food',
 'get water': 'This is a photo of get water',
 'stove/hob': 'This is a photo of stove/hob',
 'light source in kitchen': 'This is a photo of light source in kitchen',
 'drinking water': 'This is a photo of drinking water',
 'kitchen sink': 'This is a photo of kitchen sink',
 'bathroom privacy': 'This is a photo of bathroom privacy',
 'backyard': 'This is a photo of backyard',
 'front door': 'This is a photo of front door',
 'home': 'This is a photo of home',
 'source of cool': 'This is a photo of source of cool',
 'street view': 'This is a photo of street view',
 'teeth': 'This is a photo of teeth',
 'hand back': 'This is a photo of hand back',
 'hand palm': 'T

In [34]:
english_dict_topic2imgGT = {}

for topic in topics:
    key = topic2english[topic]
    english_dict_topic2imgGT[key] = dict_topic2img[topic]
print(len(english_dict_topic2imgGT))

270


In [35]:
# import torch
# import gc

# torch.cuda.empty_cache()
# gc.collect()

## Get image embeddings

In [38]:
# Load model directly
from transformers import AutoProcessor, AutoModel

model, transform = create_model_from_pretrained("nllb-clip-large-siglip", "v1", device="cuda")
model = model.eval()

In [39]:
from open_clip import get_tokenizer

tokenizer = get_tokenizer("nllb-clip-large-siglip")

In [40]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
path_images = [path_data + s for s in list(data['imageRelPath'])]
image_ids1 = [s.split("/")[9] for s in path_images]
image_ids = data['id']
assert(set(image_ids1) == set(image_ids))
# path_images_out = [path_data + img_id + "npy" for img_id in image_ids]
path_images[:2]

In [42]:
len(path_images)

36754

In [46]:
def load_image(url_or_path):
    if url_or_path.startswith("http://") or url_or_path.startswith("https://"):
        return Image.open(requests.get(url_or_path, stream=True).raw)
    else:
        return Image.open(url_or_path)


In [43]:
image_path_list = []
imgs_corrupted = []
for image_path in tqdm.tqdm(path_images):
        path_to_img = image_path
        if Path(path_to_img).is_file():
            image_path_list.append(path_to_img)
        else:
            imgs_corrupted.append(path_to_img)
len(image_path_list)

100%|██████████| 36754/36754 [22:01<00:00, 27.82it/s]


36754

In [44]:
len(imgs_corrupted)

0

In [76]:
count = 1000
end = len(image_path_list) // count + 1
print(end)

list_img_emb = []
image_names_list = []
for k in range(0, end):
    print(k)                    
    imgs_corrupted = []
    images_prep = []
    k_images = image_path_list[count*k:count*(k+1)]
    print(len(k_images))
    
    
    images = [load_image(img) for img in k_images]
    list_of_tensors = [transform(image).unsqueeze(0) for image in images]
    image_inputs = torch.stack(list_of_tensors, dim=0)
    image_inputs = image_inputs.to("cuda")
    
    
    with torch.no_grad():
        image_feature = [model.encode_image(image_input) for image_input in image_inputs]
        image_features = torch.stack(image_feature, dim=0)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        image_features = image_features.squeeze(1)
    
    
    #img_emb = img_model.encode([Image.open(filepath) for filepath in k_images], batch_size=32, convert_to_tensor=True, show_progress_bar=True)
    torch.cuda.empty_cache()
    list_img_emb.append(image_features)
    
    image_names = [image_path.split("/")[-1].split(".")[0] for image_path in image_path_list]
    image_names_list.append(image_names)
    
print("Image encoding completed..")

37
0
1000


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

1
1000


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

2
1000


...

35
1000


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

36
754


Batches:   0%|          | 0/24 [00:00<?, ?it/s]

Image encoding completed..


In [ ]:
img_features = torch.cat(list_img_emb, dim=0)
img_features.shape

In [ ]:
def flatten_extend(matrix):
    flat_list = []
    for row in matrix:
        flat_list.extend(row)
    return flat_list

In [ ]:
names = flatten_extend(image_names_list)

In [ ]:
names[0]

In [ ]:
import pickle

In [ ]:
f = open("nllb_img_embedding.pkl","wb")

# write the python object (dict) to pickle file
pickle.dump(img_features,f)

# close file
f.close()